In [ ]:
import pandas as pd
import seaborn as sns
import os
import numpy as np
from sklearn.metrics import auc

In [ ]:
HC_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/HC/basic_graph_metrics.csv'
ST01_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST01/basic_graph_metrics.csv'
ST02_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST02/basic_graph_metrics.csv'
ST03_path = '/Users/jk1/temp/stroke_resilience/output/stroke_resilience/ST03/basic_graph_metrics.csv'

output_save_path = '/Users/jk1/temp/stroke_resilience/output/analysis'

In [ ]:
HC_metrics = pd.read_csv(HC_path)
HC_metrics['timepoint_group'] = 'HC01'
HC_metrics['subject'] -= 1

ST01_metrics = pd.read_csv(ST01_path)
ST01_metrics['timepoint_group'] = 'ST01'
ST01_metrics['subject'] += 15

ST02_metrics = pd.read_csv(ST02_path)
ST02_metrics['timepoint_group'] = 'ST02'
ST02_metrics['subject'] += 15

ST03_metrics = pd.read_csv(ST03_path)
ST03_metrics['timepoint_group'] = 'ST03'
ST03_metrics['subject'] += 15

all_groups_metrics = pd.concat([HC_metrics, ST01_metrics, ST02_metrics, ST03_metrics], ignore_index=True)

In [ ]:
# Drop threshold < 0.3
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top0']
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top10']
all_groups_metrics = all_groups_metrics[all_groups_metrics.threshold != 'top20']

In [ ]:
# use auc only over predefined area of thresholds
def custom_auc(values_over_thresholds, minimum_connectivity_threshold = 0.3):
    # only analyse thresholds above minimum_connectivity_threshold
    connectivity_thresholds = np.arange(minimum_connectivity_threshold, 1.1, 0.1)

    return auc(connectivity_thresholds,
                values_over_thresholds)


In [ ]:
edge_count_auc_df = pd.DataFrame([])
for timepoint_group in all_groups_metrics.timepoint_group.unique():
    timepoint_group_df = all_groups_metrics[all_groups_metrics['timepoint_group'] == timepoint_group]
    for subject in timepoint_group_df.subject.unique():
        subject_df = timepoint_group_df[timepoint_group_df['subject'] == subject]
        subject_edge_count_auc = custom_auc(np.array(subject_df['n_edges']))
        edge_count_auc_df = edge_count_auc_df.append({'subject': subject,
                                                       'timepoint': timepoint_group.split('0')[1],
                                                      'group': timepoint_group.split('0')[0],
                                                       'edge_count_auc': subject_edge_count_auc},
                                                      ignore_index=True)

In [ ]:
edge_count_auc_df.to_csv(os.path.join(output_save_path, 'edge_count_auc.csv'))

Correct for missing values manually
- TP1: patient 16
- TP2: patient 20
- TP3: patient 28
